In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

In [3]:
collections = client.get_collections()
print(collections)

collections=[CollectionDescription(name='test'), CollectionDescription(name='cleaned_articles')]


In [4]:
client.get_collection("cleaned_articles")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, warnings=None, indexed_vectors_count=0, points_count=76, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors={}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0, wal_retain_closed=1), quantization_config=None, strict_mode_config=None, metadata=None), payload_schema={})

In [8]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(collection_name='test',
                          vectors_config=VectorParams(size=4, distance=Distance.DOT),)

True

In [10]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name='test',
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


In [14]:
search_result = client.query_points(
    collection_name='test',
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=True,
    with_vectors=True,
    limit=3,
)
print(search_result)

points=[ScoredPoint(id=4, version=1, score=1.362, payload={'city': 'New York'}, vector=[0.18, 0.01, 0.85, 0.8], shard_key=None, order_value=None), ScoredPoint(id=1, version=1, score=1.273, payload={'city': 'Berlin'}, vector=[0.05, 0.61, 0.76, 0.74], shard_key=None, order_value=None), ScoredPoint(id=3, version=1, score=1.208, payload={'city': 'Moscow'}, vector=[0.36, 0.55, 0.47, 0.94], shard_key=None, order_value=None)]


In [15]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result2 = client.query_points(
    collection_name='test',
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=FieldCondition(key="city", match=MatchValue(value='London'))
    ),
    with_payload=True,
    with_vectors=True,
    limit=3
)
print(search_result2)

points=[ScoredPoint(id=2, version=1, score=0.871, payload={'city': 'London'}, vector=[0.19, 0.81, 0.75, 0.11], shard_key=None, order_value=None)]


In [6]:
find_results, _ = client.scroll(
    collection_name='test',

)
for result in find_results:
    print(result)

id='ce8a8d68-27c9-420e-9496-907428c74cc9' payload={'city': 'Wuhan'} vector=None shard_key=None order_value=None


In [5]:
from qdrant_client import models
client.delete(
    collection_name='test',
    points_selector=models.PointIdsList(points=[1,2,3,4,5,6])
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)